In [ ]:
# default_exp train

In [13]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline

以 所有phase数据中 在test中的user_id的最后一次点击作为 线下验证集

这样做的好处是尽量减少训练的时间

val要打印每个phase的评分，总评分，总评分/phase_num

In [48]:
# export
import os
import sys
sys.path.append('../nbdevlib/')
from algo.rs.matrix import *
from code.val import main as val
from code.eda import * 
import argparse
import pandas as pd  
import numpy as np
from tqdm import tqdm  
from collections import defaultdict  
import math  
from sklearn.metrics.pairwise import cosine_similarity
from loguru import logger

trace = logger.add('runtime.log')  

# recall methods

In [61]:
# export

# topk=100

def recall_from_user_item_matr(sim_item_corr, topk, user_pred_list):
    rs = pd.Series()
    for u in tqdm(user_pred_list):
        items = items_list[u][-10:]
        len_items = len(items)
#         print(len_items)
        sr = pd.Series()
        for i, item in enumerate(items):
            sr = sr.append(sim_item_corr[item].sort_values()[-100:] * (len_items+i))
        sr = sr.reset_index()
        sr.columns = ['item_id', 'score']

        sr = sr.groupby('item_id')['score'].sum().reset_index()
        
        sr[~sr.item_id.isin(items_list[u])]
        rs.loc[u] = sr.sort_values('score', ascending=False)[:topk]['item_id'].to_list()
#                     break
    return rs

def recall_from_cocur_matr(matr, topk, user_pred_list):
    rs = pd.Series()
    for u in tqdm(user_pred_list):
        items = items_list[u][-20:]
        len_items = len(items)
#         print(len_items)
        sr = pd.Series()
        for i, item in enumerate(items):
            sr = sr.append(pd.Series(matr[item2id[item]].toarray()[0]).sort_values()[-100:] * (len_items+i))
        sr = sr.reset_index()
        sr.columns = ['id', 'score']

        sr = sr.groupby('id')['score'].sum().reset_index()
        sr['item_id'] = sr['id'].map(id2item)
        
        sr[~sr.item_id.isin(items_list[u])]
        rs.loc[u] = sr.sort_values('score', ascending=False)[:topk]['item_id'].to_list()
#                     break
    return rs

def recall_from_bipartite_network(df, user_col, item_col):  
    user_item_ = df.groupby(user_col)[item_col].agg(set).reset_index()  
    user_item_dict = dict(zip(user_item_[user_col], user_item_[item_col]))  
    
    item_user_ = whole_click.groupby(item_col)[user_col].agg(set).reset_index()  
    item_user_dict = dict(zip(item_user_[item_col], item_user_[user_col]))    

    item_cnt = defaultdict(int)  
    for user, items in tqdm(user_item_dict.items()):  
        for i in items:  
            item_cnt[i] += 1  

    sim_item = {}

    for item, users in tqdm(item_user_dict.items()):
    
        sim_item.setdefault(item, {}) 
    
        for u in users:
        
            tmp_len = len(user_item_dict[u])
        
            for relate_item in user_item_dict[u]:
                sim_item[item].setdefault(relate_item, 0)
                sim_item[item][relate_item] += 1/ (math.log(len(users)+1) * math.log(tmp_len+1))

    def recommend(sim_item_corr, user_item_dict, user_id, top_k):  
        rank = {}  
        interacted_items = user_item_dict[user_id]  
        for i in interacted_items:  
            for j, wij in sorted(sim_item_corr[i].items(), key=lambda d: d[1], reverse=True)[0:100]:  
                if j not in interacted_items:  
                    rank.setdefault(j, 0)  
                    rank[j] += wij  
        return sorted(rank.items(), key=lambda d: d[1], reverse=True)[:top_k]  

    item_sim_list, user_item = recall_from_bipartite_network(whole_click_train, 'user_id', 'item_id')
    recom_item = []        
    for i in tqdm(test_qtime['user_id'].unique()):  
        rank_item = recommend(item_sim_list, user_item, i, args.topk)  
        for j in rank_item:  
            recom_item.append([i, j[0], j[1]])  # user_id, item_id, sim                
    
    dfr = pd.DataFrame(recom_item)

    dfr.columns = 'user_id item_id score'.split()

    srr = dfr.sort_values('score', ascending=False).groupby('user_id')['item_id'].agg(list)
            
    return srr, sim_item, user_item_dict  

# val

In [80]:
def val(sr):
    """
    :sr: pd.Series
        user_id
        1        [52766, 17359, 8142, 114592, 82010, 102105, 11...
        2        [111215, 4207, 49803, 5291, 68704, 113633, 712...
    """
    r_list = []

    pred_num = whole_click_val.shape[0]
    for line in whole_click_val.itertuples():
        try:
            i = sr[line.user_id].index(line.item_id)
        except:
            i = 99999

        r_list.append(i)
    r_series = pd.Series(r_list)
    for i in range(50, args.topk+1, 50):
        logger.info(f'recall{i}:{r_series[r_series<i].shape[0]/pred_num}')
        
    r_series[r_series<9999].plot(kind='box', title='pp', rot=45)

# parameters

先把整个流程调通，就可以调节超参了！

In [58]:
# export

parser = argparse.ArgumentParser(description='t')
parser.add_argument('--now_phase', type=int, default=6, help='')
parser.add_argument('--alpha', type=float, default=0, help='已经买过的item权重衰减系数')
parser.add_argument('--time_decay', type=float, default=7/8, help='时间衰减')
parser.add_argument('--submit_fp', type=str, default='/Users/luoyonggui/Downloads/baseline1_itemcf3.csv', help='提交文件生成位置')
parser.add_argument('--mode', type=str, default='train', help='train test')
parser.add_argument('--topk', type=int, default=200, help='每种召回策略召回的样本数')

args = parser.parse_args(args=[])
logger.info(args)

2020-05-18 14:50:50.387 | INFO     | __main__:<module>:12 - Namespace(alpha=0, mode='train', now_phase=6, submit_fp='/Users/luoyonggui/Downloads/baseline1_itemcf3.csv', time_decay=0.875, topk=200)


In [47]:
args.now_phase

6

# load data

In [60]:
# export

whole_click_train, whole_click_val, test_qtime = load_whole_click_data(args.now_phase, BASE_DIR)

# 完整的click数据
whole_click = whole_click_train.append(whole_click_val)

whole_click.shape

2020-05-18 14:53:32.154 | INFO     | code.eda:load_whole_click_data:35 - phase: 0
2020-05-18 14:53:32.409 | INFO     | code.eda:load_whole_click_data:35 - phase: 1
2020-05-18 14:53:32.674 | INFO     | code.eda:load_whole_click_data:35 - phase: 2
2020-05-18 14:53:32.984 | INFO     | code.eda:load_whole_click_data:35 - phase: 3
2020-05-18 14:53:33.229 | INFO     | code.eda:load_whole_click_data:35 - phase: 4
2020-05-18 14:53:33.538 | INFO     | code.eda:load_whole_click_data:35 - phase: 5
2020-05-18 14:53:33.846 | INFO     | code.eda:load_whole_click_data:35 - phase: 6
2020-05-18 14:53:39.387 | INFO     | code.eda:load_whole_click_data:61 - whole_click_train: (891880, 4), whole_click_val: (12081, 4), test_qtime: (12081, 2)


(903961, 4)

In [7]:
# export
if args.mode == 'train':
    items_list = whole_click_train.groupby('user_id')['item_id'].agg(list)
    df_ = whole_click_train.copy()
elif args.mode == 'test':
    items_list = whole_click.groupby('user_id')['item_id'].agg(list)
    df_ = whole_click.copy()
    
logger.info('recall_from_cocur_matr')
save_dir=f'matrix_{args.mode}'
if not os.path.exists(save_dir):
    train_data_matrix, id2item, item2id = build_co_occurance_matrix(items_list.tolist(), window=20, 
                                                      penalty1=True, penalty2=True,
                                                      penalty3=0.9, save_dir=f'matrix_{args.mode}')
else:
    train_data_matrix, id2item, item2id = load_co_occurance_matrix(save_dir=f'matrix_{args.mode}', 
                                                      penalty1=True, penalty2=True,
                                                      penalty3=0.9)
sr1 = recall_from_cocur_matr(train_data_matrix, args.topk, test_qtime.user_id.tolist())

sr2 = recall_from_bipartite_network(df_, 'user_id', 'item_id')
if args.mode == 'train':
    val(sr1)
elif args.mode == 'test':
    sub = pd.DataFrame()

    for i in range(50):
        sub[f'c{i}'] = sr.str.get(i) 
    
    sub.to_csv(args.submit_fp, header=False)

## user_item_matrix

In [16]:
train_data_matrix, id2user, user2id, id2item, item2id = build_user_item_matrix(whole_click_train, 'user_id', 'item_id')

n_users: 31525, n_items: 98769


## cosine_matrix

In [ ]:
# 计算需要计算的item_id(必须减少item_id, 不然会爆掉)

In [10]:
items_ = []
for i, v in items_list.items():
    if i in test_qtime.user_id.tolist():
        items_.extend(v[-10:])
items_ = list(set(items_))

In [11]:
len(items_)

56028

In [18]:
tt = [item2id[i] for i in items_] 

In [20]:
t = cosine_similarity(train_data_matrix.T, train_data_matrix[:, tt].T)

In [22]:
t.shape

(98769, 56028)

In [28]:
sim_item_corr = dict()
dft = pd.DataFrame(t)
for i in tqdm(range(dft.shape[1])):
    sim_item_corr[id2item[tt[i]]] = pd.Series({id2item[ii]: v 
                                    for ii, v in dft.iloc[:, i].sort_values(ascending=False).iloc[1:101].items()})
   

100%|██████████| 56028/56028 [20:02<00:00, 46.60it/s]


# matching

## recall_from_cocur_matr

In [42]:
sr = recall_from_user_item_matr(sim_item_corr, topk, test_qtime.user_id.tolist())

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
  0%|          | 0/12081 [00:00<?, ?it/s]/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
100%|██████████| 12081/12081 [02:43<00:00, 73.98it/s]


In [85]:
sub

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15,c16,c17,c18,c19,c20,c21,c22,c23,c24,c25,c26,c27,c28,c29,c30,c31,c32,c33,c34,c35,c36,c37,c38,c39,c40,c41,c42,c43,c44,c45,c46,c47,c48,c49
11,8052,823,28210,23323,20317,78804,64689,22113,97305,75757,43074,29755,66082,64404,88058,32695,19508,27966,55959,22640,95867,21517,9555,53059,3295,52138,3777,65778,53384,32531,4089,20417,41136,107410,34832,5977,99219,3140,40159,51768,66299,110828,1154,16524,36409,48492,85475,35419,14708,90818
22,51765,60556,51359,35498,101862,32148,628,63202,45800,63428,33696,4492,95549,49582,4916,53664,1762,4534,18815,34443,57124,15169,27661,6465,35308,28348,87214,53889,77256,58674,3761,78700,376,74723,4072,28049,61180,46572,4686,49525,48177,39544,7965,313,56104,15956,380,47163,13702,11735
44,42253,28257,64650,13536,57991,25473,25164,9331,84056,26282,56160,71517,1609,88385,74551,25356,79761,100713,109463,82224,27161,25353,46491,114125,81193,85176,95536,88371,48180,41413,109920,50030,95768,90006,27470,77047,87154,2028,5705,106260,109170,26382,29945,19081,22534,48067,26062,41370,4292,70650
55,15351,3476,42130,51356,59836,57850,108054,67275,50276,31487,8132,17304,103612,6616,78499,110631,106996,82015,46186,15620,54135,19972,86261,20971,1425,7161,70650,67184,98502,23430,20994,106364,68341,49523,57366,3294,16725,39926,104452,87288,91974,39128,98171,63571,35571,113817,86903,40070,11273,91206
66,23748,66752,13224,24923,48591,53293,85158,22404,23613,85159,114763,93353,20772,77842,115018,43872,70699,98953,43211,87420,95366,30100,24818,20773,66868,10676,27879,29970,71492,39622,104108,32094,7924,43836,67607,10716,99711,82194,109650,76806,89808,45716,78914,22904,30082,93312,26758,78342,110185,35469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35393,97330,78323,40306,39704,41825,100834,38247,45118,106902,92748,27911,46536,102316,23467,91555,97139,75949,59745,81228,97349,43575,89855,92796,80278,16091,101512,61523,1289,13723,61858,68759,27229,76255,80616,60546,9613,103745,24243,39679,4535,37140,4542,64527,53239,86611,63087,107462,84380,25421,37280
35404,540,51182,95483,26758,87609,75460,74826,93024,16107,26344,8721,88274,94852,6492,57750,21893,57277,35361,82194,36127,95536,69608,70844,19706,27211,70705,43316,29975,61025,10145,36708,43836,18097,13652,87240,111297,100556,5989,29872,47878,2477,36744,110719,48650,10528,27009,62422,26465,35452,18668
35415,99289,43097,1487,81723,75977,7221,64649,88442,42071,6825,12648,43766,73132,36839,46746,595,30415,114125,12061,1140,33205,71955,20177,36495,90479,64264,38072,26368,107367,11837,67004,48281,14122,95887,12264,100394,69182,89017,48180,78779,27511,51905,20066,29301,81898,110905,19442,24164,70881,9812
35426,45647,75376,74498,15681,17213,23911,60733,89709,112185,110207,20179,26031,99328,32155,73723,28793,80091,32851,18266,22636,3870,21281,104649,76751,29514,63340,1760,72574,82951,58064,49863,37233,33810,17282,73111,90404,60984,100213,80897,17248,78841,74925,6461,2578,93295,90458,47317,101826,112044,46118


In [44]:
r_list = []
for line in whole_click_val.itertuples():
    if line.item_id in sr[line.user_id]:
        r_list.append(1)
    else:
        r_list.append(0)

print(pd.Series(r_list).mean())

0.13277046602102474


In [45]:
r_list = []
for line in whole_click_val.itertuples():
    if line.item_id in sr[line.user_id][:50]:
        r_list.append(1)
    else:
        r_list.append(0)

print(pd.Series(r_list).mean())

0.0792980713517093


In [66]:
r_list = []
for line in whole_click_val.itertuples():
    if line.item_id in sr[line.user_id][:50]:
        r_list.append(1)
    else:
        r_list.append(0)

print(pd.Series(r_list).mean())

0.06506083933449218


In [57]:
from nbdev.export import *
notebook2script()
notebook2script('3_model_new.ipynb')

Converted 3_model_new.ipynb.


In [60]:
# def get_sim_item(df, user_col, item_col, use_iif=False, mode=args.version):  
#     if mode=='qyxs':
#         user_item_ = df.groupby(user_col)[item_col].agg(set).reset_index()  
#         user_item_dict = dict(zip(user_item_[user_col], user_item_[item_col]))  

#         sim_item = {}  
#         item_cnt = defaultdict(int)  
#         for user, items in tqdm(user_item_dict.items()):  
#             for i in items:  
#                 item_cnt[i] += 1  
#                 sim_item.setdefault(i, {})  
#                 for relate_item in items:  
#                     if i == relate_item:  
#                         continue  
#                     sim_item[i].setdefault(relate_item, 0)  
#                     if not use_iif:  
#                         sim_item[i][relate_item] += 1  
#                     else:  
#                         sim_item[i][relate_item] += 1 / math.log(1 + len(items))  
#         sim_item_corr = sim_item.copy()  
#         for i, related_items in tqdm(sim_item.items()):  
#             for j, cij in related_items.items():  
#                 sim_item_corr[i][j] = cij/math.sqrt(item_cnt[i]*item_cnt[j])  

#         return sim_item_corr, user_item_dict  
  
    
#     elif mode==2:
#         user_item_ = df.groupby(user_col)['item_time'].agg(list).reset_index()  
#         user_item_dict = dict(zip(user_item_[user_col], user_item_['item_time']))  

#         sim_item = {}  
#         item_cnt = defaultdict(int)  
#         for user, items in tqdm(user_item_dict.items()):  
#             for i, time_ in items:  
#                 item_cnt[i] += 1  
#                 sim_item.setdefault(i, {})  
#                 for relate_item, relate_time_ in items:  
#                     if i == relate_item:  
#                         continue  
#                     sim_item[i].setdefault(relate_item, 0)  
#                     if not use_iif:  
#                         sim_item[i][relate_item] += 1  
#                     else:  
#                         sim_item[i][relate_item] += 1 / (math.log(1 + len(items)) * math.exp(math.fabs(relate_time_-time_)*args.alpha))  
#         sim_item_corr = sim_item.copy()  
#         for i, related_items in tqdm(sim_item.items()):  
#             for j, cij in related_items.items():  
#     #             sim_item_corr[i][j] = cij/math.sqrt(item_cnt[i]*item_cnt[j]) 
#                 sim_item_corr[i][j] = cij/(np.log(item_cnt[i]+1) * np.log(item_cnt[j]+1)) 
#     #             sim_item_corr[i][j] = cij/np.log((item_cnt[i]+1)*(item_cnt[j]+1)) 
  
#     return sim_item_corr, user_item_dict  


# def recommend(whole_click_train_, click_test_val_, sim_item_corr, user_item_dict, user_id, top_k, item_num, weight_mode='linear'):  
#     rank = {}  
#     # 该user_id购买过的items
#     dft = whole_click_train_[whole_click_train_.user_id == user_id].sort_values('time').drop_duplicates('item_id', keep='last')
#     pred_time = click_test_val_.loc[click_test_val_.user_id==user_id, 'time'].iloc[0]
#     #     if weight_mode=='linear':
# #         dft['t'] = range(dft.shape[0], dft.shape[0] * 2)
# #         dft['t'] = dft['t'] - int(dft.shape[0] * args.time_decay)

# #     elif weight_mode=='exp':
# #         dft['t'] = ((dft['time'] - 0.9837) * 10000).map(math.exp) # 最大值是最小值的15倍左右，score:0.1275
        
#     interacted_items = dft['item_time'].tolist()    
# #     weights = dft['t'].tolist()
    
#     cnt = 0
#     for i, t in interacted_items:  
#         for j, wij in sorted(sim_item_corr[i].items(), key=lambda d: d[1], reverse=True)[0:top_k]:  
#             alpha = 0.5
#             if j not in interacted_items:  
#                 alpha = 1
#             if j not in rank:
#                 rank[j] = 0
#             rank[j] += wij * alpha / math.exp(math.fabs(pred_time-t))
# #             print(math.exp(math.fabs(pred_time-t)*10000))
#         cnt += 1
#     return sorted(rank.items(), key=lambda d: d[1], reverse=True)[:item_num]  


def get_sim_item(df, user_col, item_col, use_iif=False):  
    user_item_ = df.groupby(user_col)[item_col].agg(set).reset_index()  
    user_item_dict = dict(zip(user_item_[user_col], user_item_[item_col]))  
  
    sim_item = {}  
    item_cnt = defaultdict(int)  
    for user, items in tqdm(user_item_dict.items()):  
        for i in items:  
            item_cnt[i] += 1  
            sim_item.setdefault(i, {})  
            for relate_item in items:  
                if i == relate_item:  
                    continue  
                sim_item[i].setdefault(relate_item, 0)  
                if not use_iif:  
                    sim_item[i][relate_item] += 1  
                else:  
                    sim_item[i][relate_item] += 1 / math.log(1 + len(items))  
    sim_item_corr = sim_item.copy()  
    for i, related_items in tqdm(sim_item.items()):  
        for j, cij in related_items.items():  
            sim_item_corr[i][j] = cij/(np.log(item_cnt[i]+1) * np.log(item_cnt[j]+1))
#             sim_item_corr[i][j] = cij/math.sqrt(item_cnt[i]*item_cnt[j])  
  
    return sim_item_corr, user_item_dict  
  
# ttt = train_item_df.item_id.tolist()
ttt = pd.Series(range(train_item_df.shape[0]), index = train_item_df.item_id)
def recommend(sim_item_corr, user_item_dict, user_id, top_k, item_num):  
    rank = {}  
    interacted_items = user_item_dict[user_id]  
    
    for i in interacted_items:  
        for j, wij in sorted(sim_item_corr[i].items(), key=lambda d: d[1], reverse=True)[0:top_k]:  
            if j not in interacted_items: 
                if j not in ttt:  
                    rank.setdefault(j, 0)  
                    rank[j] += wij  
    return sorted(rank.items(), key=lambda d: d[1], reverse=True)[:item_num]  
  

# fill user to 50 items  
def get_predict(df, pred_col, top_fill):  
    top_fill = [int(t) for t in top_fill.split(',')]  
    scores = [-1 * i for i in range(1, len(top_fill) + 1)]  
    ids = list(df['user_id'].unique())  
    fill_df = pd.DataFrame(ids * len(top_fill), columns=['user_id'])  
    fill_df.sort_values('user_id', inplace=True)  
    fill_df['item_id'] = top_fill * len(ids)  
    fill_df[pred_col] = scores * len(ids)  
    df = df.append(fill_df)  
    df.sort_values(pred_col, ascending=False, inplace=True)  
    df = df.drop_duplicates(subset=['user_id', 'item_id'], keep='first')  
    df['rank'] = df.groupby('user_id')[pred_col].rank(method='first', ascending=False)  
    df = df[df['rank'] <= 50]  
    df = df.groupby('user_id')['item_id'].apply(lambda x: ','.join([str(i) for i in x])).str.split(',', expand=True).reset_index()  
    return df  

# main

In [31]:
def main(whole_click_train, submit_fname):
    import time
    t0 = time.time()
    recom_item = []  
    item_sim_list, user_item = get_sim_item(whole_click_train, 'user_id', 'item_id', use_iif=True)  

    for i in tqdm(click_test_val['user_id'].unique()):  
#         rank_item = recommend(whole_click_train, click_test_val, item_sim_list, user_item, i, 200, 50) 
        rank_item = recommend(item_sim_list, user_item, i, 200, 50) 
        for j in rank_item:  
            recom_item.append([i, j[0], j[1]])  

    # find most popular items  
    top50_click = whole_click_train['item_id'].value_counts().index[:50].values  
    top50_click = ','.join([str(i) for i in top50_click])  
    recom_df = pd.DataFrame(recom_item, columns=['user_id', 'item_id', 'sim'])  
    result = get_predict(recom_df, 'sim', top50_click)  
    result.to_csv(submit_fname, index=False, header=None)
#     result1 = result.set_index('user_id')


#     result1 = result1.applymap(float).applymap(int)
#     result1['p'] = result1.values.tolist()
#     print(evaluate_each_phase(result1['p'].to_dict(), answers))

    t1 = time.time()
    logger.info(val(now_phase, submit_fname))
    print(f'{t1-t0}s')
    return result

In [26]:
submit_fname = '/Users/luoyonggui/Downloads/baseline1_itemcf3.csv'

In [55]:
result = main(whole_click_train, submit_fname)





  0%|          | 0/29571 [00:00<?, ?it/s]



  1%|          | 184/29571 [00:00<00:15, 1838.77it/s]



  1%|          | 309/29571 [00:00<00:18, 1610.81it/s]



  1%|▏         | 432/29571 [00:00<00:20, 1440.97it/s]



  2%|▏         | 580/29571 [00:00<00:20, 1437.05it/s]



  2%|▏         | 727/29571 [00:00<00:19, 1442.83it/s]



  3%|▎         | 844/29571 [00:00<00:21, 1321.38it/s]



  3%|▎         | 963/29571 [00:00<00:22, 1278.42it/s]



  4%|▍         | 1121/29571 [00:00<00:21, 1354.71it/s]



  4%|▍         | 1282/29571 [00:00<00:19, 1422.02it/s]



  5%|▍         | 1421/29571 [00:01<00:21, 1299.02it/s]



  5%|▌         | 1582/29571 [00:01<00:20, 1378.89it/s]



  6%|▌         | 1730/29571 [00:01<00:19, 1403.71it/s]



  6%|▋         | 1872/29571 [00:01<00:22, 1226.63it/s]



  7%|▋         | 2000/29571 [00:01<00:24, 1103.58it/s]



  7%|▋         | 2116/29571 [00:01<00:24, 1102.95it/s]



  8%|▊         | 2231/29571 [00:01<00:24, 1104.78it/s]



  8%|▊         | 2345/29571 [0

  1%|          | 908/89410 [00:02<05:13, 282.32it/s]



  1%|          | 939/89410 [00:02<05:15, 280.01it/s]



  1%|          | 969/89410 [00:02<05:27, 269.87it/s]



  1%|          | 1021/89410 [00:02<04:45, 310.10it/s]



  1%|          | 1055/89410 [00:03<04:44, 310.80it/s]



  1%|▏         | 1118/89410 [00:03<04:01, 366.29it/s]



  1%|▏         | 1166/89410 [00:03<03:45, 390.88it/s]



  1%|▏         | 1210/89410 [00:03<04:00, 366.99it/s]



  1%|▏         | 1252/89410 [00:03<03:59, 368.54it/s]



  1%|▏         | 1292/89410 [00:03<04:37, 317.55it/s]



  1%|▏         | 1327/89410 [00:03<05:04, 288.84it/s]



  2%|▏         | 1371/89410 [00:03<04:33, 321.50it/s]



  2%|▏         | 1407/89410 [00:04<04:59, 293.88it/s]



  2%|▏         | 1439/89410 [00:04<05:05, 287.78it/s]



  2%|▏         | 1489/89410 [00:04<04:29, 326.46it/s]



  2%|▏         | 1529/89410 [00:04<04:15, 344.07it/s]



  2%|▏         | 1594/89410 [00:04<03:40, 397.53it/s]



  2%|▏         | 1639/89410 [00:04

 18%|█▊        | 15931/89410 [00:30<02:05, 586.44it/s]



 18%|█▊        | 15991/89410 [00:30<02:12, 555.16it/s]



 18%|█▊        | 16075/89410 [00:30<01:58, 617.68it/s]



 18%|█▊        | 16153/89410 [00:30<01:51, 654.89it/s]



 18%|█▊        | 16228/89410 [00:30<01:47, 680.67it/s]



 18%|█▊        | 16299/89410 [00:31<01:52, 649.42it/s]



 18%|█▊        | 16366/89410 [00:31<02:28, 493.54it/s]



 18%|█▊        | 16444/89410 [00:31<02:11, 554.44it/s]



 18%|█▊        | 16508/89410 [00:31<02:07, 572.40it/s]



 19%|█▊        | 16593/89410 [00:31<01:54, 634.18it/s]



 19%|█▊        | 16663/89410 [00:31<01:58, 613.65it/s]



 19%|█▊        | 16735/89410 [00:31<01:53, 641.94it/s]



 19%|█▉        | 16811/89410 [00:31<01:48, 670.83it/s]



 19%|█▉        | 16881/89410 [00:31<01:54, 632.28it/s]



 19%|█▉        | 16947/89410 [00:32<01:58, 612.65it/s]



 19%|█▉        | 17023/89410 [00:32<01:53, 639.82it/s]



 19%|█▉        | 17089/89410 [00:32<02:06, 573.19it/s]



 19%|█▉       

 40%|███▉      | 35475/89410 [00:56<00:54, 984.53it/s]



 40%|███▉      | 35578/89410 [00:56<00:54, 993.70it/s]



 40%|███▉      | 35702/89410 [00:57<00:50, 1056.29it/s]



 40%|████      | 35810/89410 [00:57<00:51, 1033.10it/s]



 40%|████      | 35916/89410 [00:57<00:57, 930.89it/s] 



 40%|████      | 36013/89410 [00:57<00:59, 898.09it/s]



 40%|████      | 36121/89410 [00:57<00:56, 940.58it/s]



 41%|████      | 36218/89410 [00:57<00:58, 903.58it/s]



 41%|████      | 36311/89410 [00:57<00:58, 906.40it/s]



 41%|████      | 36405/89410 [00:57<00:57, 915.02it/s]



 41%|████      | 36513/89410 [00:57<00:55, 958.94it/s]



 41%|████      | 36615/89410 [00:58<00:54, 976.32it/s]



 41%|████      | 36720/89410 [00:58<00:53, 992.60it/s]



 41%|████      | 36820/89410 [00:58<00:54, 971.45it/s]



 41%|████▏     | 36923/89410 [00:58<00:53, 987.28it/s]



 41%|████▏     | 37023/89410 [00:58<00:57, 914.62it/s]



 42%|████▏     | 37116/89410 [00:58<00:57, 909.84it/s]



 42%|████▏ 

 68%|██████▊   | 60829/89410 [01:21<00:24, 1160.64it/s]



 68%|██████▊   | 60948/89410 [01:21<00:25, 1122.84it/s]



 68%|██████▊   | 61094/89410 [01:21<00:23, 1204.37it/s]



 68%|██████▊   | 61218/89410 [01:21<00:24, 1153.41it/s]



 69%|██████▊   | 61338/89410 [01:22<00:24, 1156.74it/s]



 69%|██████▊   | 61456/89410 [01:22<00:25, 1095.02it/s]



 69%|██████▉   | 61568/89410 [01:22<00:26, 1047.16it/s]



 69%|██████▉   | 61675/89410 [01:22<00:27, 1015.49it/s]



 69%|██████▉   | 61782/89410 [01:22<00:26, 1031.05it/s]



 69%|██████▉   | 61912/89410 [01:22<00:25, 1097.92it/s]



 69%|██████▉   | 62024/89410 [01:22<00:26, 1051.09it/s]



 69%|██████▉   | 62131/89410 [01:22<00:26, 1044.10it/s]



 70%|██████▉   | 62237/89410 [01:23<00:28, 964.64it/s] 



 70%|██████▉   | 62352/89410 [01:23<00:26, 1012.95it/s]



 70%|██████▉   | 62462/89410 [01:23<00:25, 1036.68it/s]



 70%|███████   | 62587/89410 [01:23<00:24, 1088.73it/s]



 70%|███████   | 62710/89410 [01:23<00:23, 1124.13it/s]


...

 80%|████████  | 8211/10260 [01:42<00:37, 54.15it/s]



 80%|████████  | 8222/10260 [01:42<00:32, 62.64it/s]



 80%|████████  | 8232/10260 [01:42<00:31, 65.18it/s]



 80%|████████  | 8242/10260 [01:42<00:27, 72.45it/s]



 80%|████████  | 8252/10260 [01:42<00:27, 74.26it/s]



 81%|████████  | 8262/10260 [01:42<00:28, 70.70it/s]



 81%|████████  | 8270/10260 [01:43<00:30, 66.22it/s]



 81%|████████  | 8278/10260 [01:43<00:30, 65.96it/s]



 81%|████████  | 8287/10260 [01:43<00:29, 67.08it/s]



 81%|████████  | 8294/10260 [01:43<00:29, 67.31it/s]



 81%|████████  | 8302/10260 [01:43<00:27, 70.29it/s]



 81%|████████  | 8310/10260 [01:43<00:29, 65.85it/s]



 81%|████████  | 8319/10260 [01:43<00:27, 70.42it/s]



 81%|████████  | 8327/10260 [01:43<00:30, 63.22it/s]



 81%|████████  | 8334/10260 [01:44<00:31, 60.74it/s]



 81%|████████▏ | 8346/10260 [01:44<00:26, 71.04it/s]



 81%|████████▏ | 8360/10260 [01:44<00:22, 83.29it/s]



 82%|████████▏ | 8373/10260 [01:44<00:20, 91.03i

 99%|█████████▉| 10180/10260 [02:16<00:01, 65.46it/s]



 99%|█████████▉| 10188/10260 [02:16<00:01, 66.66it/s]



 99%|█████████▉| 10198/10260 [02:16<00:00, 73.60it/s]



 99%|█████████▉| 10207/10260 [02:17<00:00, 61.01it/s]



100%|█████████▉| 10215/10260 [02:17<00:00, 64.35it/s]



100%|█████████▉| 10223/10260 [02:17<00:00, 54.26it/s]



100%|█████████▉| 10230/10260 [02:17<00:00, 53.39it/s]



100%|█████████▉| 10244/10260 [02:17<00:00, 64.54it/s]



100%|██████████| 10260/10260 [02:17<00:00, 74.39it/s]
2020-05-11 17:10:48.239 | INFO     | code.val:create_underexpose_test_qtime_with_answer:208 - phase: 0
2020-05-11 17:10:48.241 | INFO     | code.val:create_underexpose_test_qtime_with_answer:211 - data_gen/underexpose_test_qtime_with_answer-0.csv exists!
2020-05-11 17:10:48.243 | INFO     | code.val:create_underexpose_test_qtime_with_answer:208 - phase: 1
2020-05-11 17:10:48.245 | INFO     | code.val:create_underexpose_test_qtime_with_answer:211 - data_gen/underexpose_test_qtime_with_a

current_time: 1589188250
date_time: 2020-05-11 17:10:50
current_phase: 5


2020-05-11 17:10:51.059 | INFO     | __main__:main:27 - ((0.0344264842569828, 0.015331625938415527, 0.020242370665073395, 0.007922083139419556), {1: array([0.00463499, 0.00232479, 0.00442478, 0.00233597], dtype=float32), 2: array([0.00650888, 0.0031376 , 0.0035461 , 0.00112552], dtype=float32), 3: array([0.00358209, 0.00160517, 0.0011655 , 0.0003885 ], dtype=float32), 4: array([0.00468384, 0.00200594, 0.00340522, 0.00188655], dtype=float32), 5: array([0.01501669, 0.00625812, 0.00770077, 0.00218554], dtype=float32)})


user_id 30415 of phase 0 not in submission
error occurred during evaluation
phase_id: 1, score: [0.00463499 0.00232479 0.00442478 0.00233597]
phase_id: 2, score: [0.00650888 0.0031376  0.0035461  0.00112552]
phase_id: 3, score: [0.00358209 0.00160517 0.0011655  0.0003885 ]
phase_id: 4, score: [0.00468384 0.00200594 0.00340522 0.00188655]
phase_id: 5, score: [0.01501669 0.00625812 0.00770077 0.00218554]
267.9403557777405s


In [36]:
result = main(whole_click_train, submit_fname)

100%|██████████| 10260/10260 [02:30<00:00, 68.23it/s]
2020-05-11 16:25:47.476 | INFO     | code.val:create_underexpose_test_qtime_with_answer:208 - phase: 0
2020-05-11 16:25:47.478 | INFO     | code.val:create_underexpose_test_qtime_with_answer:211 - data_gen/underexpose_test_qtime_with_answer-0.csv exists!
2020-05-11 16:25:47.480 | INFO     | code.val:create_underexpose_test_qtime_with_answer:208 - phase: 1
2020-05-11 16:25:47.482 | INFO     | code.val:create_underexpose_test_qtime_with_answer:211 - data_gen/underexpose_test_qtime_with_answer-1.csv exists!
2020-05-11 16:25:47.484 | INFO     | code.val:create_underexpose_test_qtime_with_answer:208 - phase: 2
2020-05-11 16:25:47.486 | INFO     | code.val:create_underexpose_test_qtime_with_answer:211 - data_gen/underexpose_test_qtime_with_answer-2.csv exists!
2020-05-11 16:25:47.487 | INFO     | code.val:create_underexpose_test_qtime_with_answer:208 - phase: 3
2020-05-11 16:25:47.490 | INFO     | code.val:create_underexpose_test_qtime_wi

current_time: 1589185550
date_time: 2020-05-11 16:25:50
current_phase: 5


2020-05-11 16:25:50.549 | INFO     | __main__:main:27 - ((0.13446837663650513, 0.05077897012233734, 0.11830619722604752, 0.040628671646118164), {0: array([0.0288635 , 0.01210867, 0.02608696, 0.01086523], dtype=float32), 1: array([0.01680185, 0.00727749, 0.0210177 , 0.0074088 ], dtype=float32), 2: array([0.01538462, 0.00605695, 0.00945626, 0.00254042], dtype=float32), 3: array([0.01373134, 0.00585497, 0.01398601, 0.00492104], dtype=float32), 4: array([0.014637  , 0.00432228, 0.01475596, 0.00413872], dtype=float32), 5: array([0.04505005, 0.01515861, 0.0330033 , 0.01075446], dtype=float32)})


phase_id: 0, score: [0.0288635  0.01210867 0.02608696 0.01086523]
phase_id: 1, score: [0.01680185 0.00727749 0.0210177  0.0074088 ]
phase_id: 2, score: [0.01538462 0.00605695 0.00945626 0.00254042]
phase_id: 3, score: [0.01373134 0.00585497 0.01398601 0.00492104]
phase_id: 4, score: [0.014637   0.00432228 0.01475596 0.00413872]
phase_id: 5, score: [0.04505005 0.01515861 0.0330033  0.01075446]
285.09283685684204s


In [61]:
result = main(whole_click, submit_fname)





  0%|          | 0/29571 [00:00<?, ?it/s]



  1%|          | 184/29571 [00:00<00:16, 1810.01it/s]



  1%|          | 316/29571 [00:00<00:18, 1619.16it/s]



  2%|▏         | 463/29571 [00:00<00:18, 1568.02it/s]



  2%|▏         | 624/29571 [00:00<00:18, 1559.55it/s]



  3%|▎         | 756/29571 [00:00<00:19, 1478.46it/s]



  3%|▎         | 897/29571 [00:00<00:19, 1443.17it/s]



  4%|▎         | 1048/29571 [00:00<00:19, 1433.29it/s]



  4%|▍         | 1226/29571 [00:00<00:18, 1517.79it/s]



  5%|▍         | 1378/29571 [00:00<00:18, 1494.30it/s]



  5%|▌         | 1522/29571 [00:01<00:19, 1414.24it/s]



  6%|▌         | 1681/29571 [00:01<00:19, 1454.87it/s]



  6%|▌         | 1825/29571 [00:01<00:20, 1336.06it/s]



  7%|▋         | 1960/29571 [00:01<00:21, 1292.25it/s]



  7%|▋         | 2090/29571 [00:01<00:21, 1250.02it/s]



  7%|▋         | 2216/29571 [00:01<00:22, 1207.02it/s]



  8%|▊         | 2338/29571 [00:01<00:22, 1196.68it/s]



  8%|▊         | 2465/29571 [

  2%|▏         | 1634/89415 [00:04<03:14, 452.33it/s]



  2%|▏         | 1683/89415 [00:04<03:13, 452.66it/s]



  2%|▏         | 1732/89415 [00:04<03:10, 461.25it/s]



  2%|▏         | 1796/89415 [00:04<02:55, 500.29it/s]



  2%|▏         | 1864/89415 [00:04<02:44, 533.77it/s]



  2%|▏         | 1920/89415 [00:04<03:03, 476.82it/s]



  2%|▏         | 1978/89415 [00:04<02:53, 503.40it/s]



  2%|▏         | 2031/89415 [00:05<02:53, 502.87it/s]



  2%|▏         | 2090/89415 [00:05<02:46, 523.28it/s]



  2%|▏         | 2144/89415 [00:05<02:54, 499.20it/s]



  2%|▏         | 2196/89415 [00:05<03:09, 460.84it/s]



  3%|▎         | 2257/89415 [00:05<02:57, 491.29it/s]



  3%|▎         | 2313/89415 [00:05<02:53, 503.11it/s]



  3%|▎         | 2365/89415 [00:05<02:53, 500.79it/s]



  3%|▎         | 2416/89415 [00:05<02:52, 503.47it/s]



  3%|▎         | 2467/89415 [00:06<03:42, 389.99it/s]



  3%|▎         | 2511/89415 [00:06<05:13, 276.80it/s]



  3%|▎         | 2582/89415 [00

 19%|█▊        | 16753/89415 [00:32<02:16, 530.64it/s]



 19%|█▉        | 16808/89415 [00:32<02:21, 514.54it/s]



 19%|█▉        | 16880/89415 [00:32<02:09, 562.03it/s]



 19%|█▉        | 16939/89415 [00:32<02:31, 477.34it/s]



 19%|█▉        | 16991/89415 [00:32<02:32, 475.98it/s]



 19%|█▉        | 17063/89415 [00:32<02:17, 524.74it/s]



 19%|█▉        | 17119/89415 [00:32<02:20, 514.89it/s]



 19%|█▉        | 17173/89415 [00:33<02:27, 490.16it/s]



 19%|█▉        | 17241/89415 [00:33<02:15, 531.12it/s]



 19%|█▉        | 17332/89415 [00:33<01:58, 606.65it/s]



 19%|█▉        | 17416/89415 [00:33<01:48, 661.51it/s]



 20%|█▉        | 17488/89415 [00:33<01:47, 669.31it/s]



 20%|█▉        | 17559/89415 [00:33<01:51, 646.11it/s]



 20%|█▉        | 17627/89415 [00:33<01:55, 622.04it/s]



 20%|█▉        | 17693/89415 [00:33<01:53, 632.32it/s]



 20%|█▉        | 17777/89415 [00:33<01:44, 682.70it/s]



 20%|█▉        | 17869/89415 [00:34<01:36, 739.20it/s]



 20%|██       

 41%|████▏     | 37042/89415 [00:58<00:53, 981.83it/s]



 42%|████▏     | 37142/89415 [00:58<00:58, 886.17it/s]



 42%|████▏     | 37234/89415 [00:58<00:58, 886.24it/s]



 42%|████▏     | 37325/89415 [00:58<01:05, 795.26it/s]



 42%|████▏     | 37411/89415 [00:58<01:03, 813.56it/s]



 42%|████▏     | 37495/89415 [00:58<01:04, 808.20it/s]



 42%|████▏     | 37578/89415 [00:58<01:05, 790.52it/s]



 42%|████▏     | 37659/89415 [00:58<01:06, 776.77it/s]



 42%|████▏     | 37739/89415 [00:59<01:06, 782.31it/s]



 42%|████▏     | 37821/89415 [00:59<01:05, 790.29it/s]



 42%|████▏     | 37901/89415 [00:59<01:10, 727.35it/s]



 42%|████▏     | 37979/89415 [00:59<01:09, 741.73it/s]



 43%|████▎     | 38055/89415 [00:59<01:16, 667.60it/s]



 43%|████▎     | 38134/89415 [00:59<01:13, 697.10it/s]



 43%|████▎     | 38217/89415 [00:59<01:10, 729.02it/s]



 43%|████▎     | 38292/89415 [00:59<01:11, 711.03it/s]



 43%|████▎     | 38382/89415 [00:59<01:07, 758.52it/s]



 43%|████▎    

 69%|██████▉   | 61661/89415 [01:23<00:26, 1043.79it/s]



 69%|██████▉   | 61769/89415 [01:23<00:27, 1000.39it/s]



 69%|██████▉   | 61875/89415 [01:23<00:27, 1017.20it/s]



 69%|██████▉   | 61979/89415 [01:23<00:27, 1006.75it/s]



 69%|██████▉   | 62085/89415 [01:23<00:26, 1019.62it/s]



 70%|██████▉   | 62188/89415 [01:23<00:28, 966.42it/s] 



 70%|██████▉   | 62286/89415 [01:23<00:28, 944.41it/s]



 70%|██████▉   | 62382/89415 [01:23<00:28, 935.95it/s]



 70%|██████▉   | 62505/89415 [01:24<00:26, 1005.92it/s]



 70%|███████   | 62626/89415 [01:24<00:25, 1056.38it/s]



 70%|███████   | 62734/89415 [01:24<00:25, 1053.81it/s]



 70%|███████   | 62870/89415 [01:24<00:23, 1128.04it/s]



 70%|███████   | 62986/89415 [01:24<00:24, 1067.37it/s]



 71%|███████   | 63096/89415 [01:24<00:25, 1019.44it/s]



 71%|███████   | 63201/89415 [01:24<00:27, 967.81it/s] 



 71%|███████   | 63329/89415 [01:24<00:24, 1043.89it/s]



 71%|███████   | 63475/89415 [01:24<00:22, 1140.83it/s]




...

 81%|████████  | 8260/10260 [01:41<00:27, 71.55it/s]



 81%|████████  | 8268/10260 [01:41<00:36, 54.89it/s]



 81%|████████  | 8277/10260 [01:42<00:34, 56.97it/s]



 81%|████████  | 8287/10260 [01:42<00:31, 61.81it/s]



 81%|████████  | 8296/10260 [01:42<00:28, 67.90it/s]



 81%|████████  | 8304/10260 [01:42<00:34, 56.73it/s]



 81%|████████  | 8313/10260 [01:42<00:30, 63.72it/s]



 81%|████████  | 8321/10260 [01:42<00:30, 63.19it/s]



 81%|████████  | 8328/10260 [01:42<00:32, 60.04it/s]



 81%|████████  | 8335/10260 [01:42<00:33, 57.19it/s]



 81%|████████▏ | 8345/10260 [01:43<00:29, 65.56it/s]



 81%|████████▏ | 8357/10260 [01:43<00:25, 74.79it/s]



 82%|████████▏ | 8372/10260 [01:43<00:21, 86.67it/s]



 82%|████████▏ | 8383/10260 [01:43<00:22, 82.03it/s]



 82%|████████▏ | 8393/10260 [01:43<00:27, 67.79it/s]



 82%|████████▏ | 8405/10260 [01:43<00:24, 77.06it/s]



 82%|████████▏ | 8414/10260 [01:43<00:24, 75.23it/s]



 82%|████████▏ | 8423/10260 [01:43<00:24, 74.87i

2020-05-11 17:28:41.892 | INFO     | code.val:create_underexpose_test_qtime_with_answer:208 - phase: 1
2020-05-11 17:28:41.894 | INFO     | code.val:create_underexpose_test_qtime_with_answer:211 - data_gen/underexpose_test_qtime_with_answer-1.csv exists!
2020-05-11 17:28:41.896 | INFO     | code.val:create_underexpose_test_qtime_with_answer:208 - phase: 2
2020-05-11 17:28:41.897 | INFO     | code.val:create_underexpose_test_qtime_with_answer:211 - data_gen/underexpose_test_qtime_with_answer-2.csv exists!
2020-05-11 17:28:41.899 | INFO     | code.val:create_underexpose_test_qtime_with_answer:208 - phase: 3
2020-05-11 17:28:41.900 | INFO     | code.val:create_underexpose_test_qtime_with_answer:211 - data_gen/underexpose_test_qtime_with_answer-3.csv exists!
2020-05-11 17:28:41.902 | INFO     | code.val:create_underexpose_test_qtime_with_answer:208 - phase: 4
2020-05-11 17:28:41.904 | INFO     | code.val:create_underexpose_test_qtime_with_answer:211 - data_gen/underexpose_test_qtime_with_a

current_time: 1589189324
date_time: 2020-05-11 17:28:44
current_phase: 5


2020-05-11 17:28:44.675 | INFO     | __main__:main:27 - ((0.0, 0.0, 0.0, 0.0), {0: array([0., 0., 0., 0.], dtype=float32), 1: array([0., 0., 0., 0.], dtype=float32), 2: array([0., 0., 0., 0.], dtype=float32), 3: array([0., 0., 0., 0.], dtype=float32), 4: array([0., 0., 0., 0.], dtype=float32), 5: array([0., 0., 0., 0.], dtype=float32)})


phase_id: 0, score: [0. 0. 0. 0.]
phase_id: 1, score: [0. 0. 0. 0.]
phase_id: 2, score: [0. 0. 0. 0.]
phase_id: 3, score: [0. 0. 0. 0.]
phase_id: 4, score: [0. 0. 0. 0.]
phase_id: 5, score: [0. 0. 0. 0.]
265.5958709716797s
